# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning

# Creación de Dataframes

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

En este notebook, que es un re-trabajo sobre el `Notebook Anexo` del [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) se buscan features del set de datos y se crean nuevos atributos a partir de ellos. Estos se almacenan en nuevos archivos `.csv` para luego ser concatenados al set de datos principal.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

df = pd.read_csv('./data/events_up_to_01062018.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [2]:
print("len: {}".format(len(df)))
df.head()

len: 2341681


,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## brands.csv

Marcas de celulares

In [3]:
df['model'].unique()

array(['Samsung Galaxy J7 Prime', 'iPhone 5s', 'iPhone 6S', 'iPhone 7',
       'LG G4 H818P', nan, 'iPhone 6', 'iPhone 6 Plus',
       'Motorola Moto G5 Plus', 'Motorola Moto G4 Plus', 'iPad Air Wi-Fi',
       'Samsung Galaxy S6 Flat', 'Samsung Galaxy S8 Plus',
       'Samsung Galaxy S5', 'iPhone SE', 'Samsung Galaxy S7',
       'Motorola Moto X2', 'Motorola Moto G3 4G',
       'Motorola Moto X Play 4G Dual', 'Motorola Moto G4 Play',
       'Samsung Galaxy J5', 'Samsung Galaxy S5 New Edition Duos ',
       'Motorola Moto G3 HDTV', 'Motorola Moto G5 ', 'Samsung Galaxy J3',
       'Samsung Galaxy S6 Edge', 'Samsung Galaxy Note 8',
       'Sony Xperia Z2', 'Motorola Moto G2 3G Dual',
       'Motorola Moto Z Play', 'Samsung Galaxy S8', 'Samsung Galaxy On 7',
       'Samsung Galaxy S7 Edge', 'iPhone 8', 'iPhone 6S Plus',
       'LG K10 Novo', 'Samsung Galaxy Core Plus Duos TV', 'iPhone 5c',
       'iPhone 4S', 'Motorola Moto X Style', 'iPhone 7 Plus',
       'Samsung Galaxy A5 2016', 'Samsu

In [4]:
checked = ['iphone', 'samsung', 'motorola', 'lenovo', 'sony', 'lg', 'ipad', 'asus', 'quantum', 'blackberry']
model_parsed = df['model'].dropna().map(lambda x: x.lower())
model_parsed = model_parsed.map(lambda x: x.split())

def find_brand(model):
    for str in model:
        if str in checked:
            return str
    return "other"


df['brand'] = model_parsed.map(find_brand)
df['brand'] = df['brand'].astype('category')

In [5]:
df[['model', 'brand']].dropna().drop_duplicates().to_csv('data/brands.csv', index=False)

In [6]:
brands_csv = pd.read_csv("data/brands.csv", low_memory=False)
display(brands_csv.head())
display(brands_csv.shape)

,model,brand
0,Samsung Galaxy J7 Prime,samsung
1,iPhone 5s,iphone
2,iPhone 6S,iphone
3,iPhone 7,iphone
4,LG G4 H818P,lg


(208, 2)

## os.csv

Sistemas operativos de celulares

In [7]:
df['operating_system_version'].unique()

array([nan, 'Android 5.0.2', 'Ubuntu ', 'Android 7', 'Android 6.0.1',
       'Windows 7 ', 'Windows 10 ', 'iOS 11.0.3', 'Android 6',
       'Android 4.4.4', 'Android 7.1.1', 'Mac OS X 10.12.6',
       'Android 5.1', 'Windows 8.1 ', 'Android 5.1.1', 'Android 8.1',
       'Windows 8 ', 'iOS 9.3.5', 'Android 4.2.2', 'Android 5',
       'iOS 11.3', 'Android 4.1.2', 'Android 4.4.2', 'Android 5.0.1',
       'iOS 11.1.1', 'Windows XP ', 'iOS 10.3.3', 'Windows Phone 8.1',
       'Chrome OS 10452.85', 'Android 8', 'Mac OS X 10.10.4',
       'iOS 11.2.6', 'Android ', 'Android 4.3', 'Mac OS X 10.11.6',
       'Windows Vista ', 'iOS 11.1.2', 'Fedora ', 'Windows Phone 10',
       'Linux ', 'Mac OS X 10.13.4', 'Android 7.1.2', 'iOS 8.1.3',
       'iOS 11.2.1', 'Android 4.0.3', 'FreeBSD ', 'iOS 11.2.2',
       'Android 2.3.6', 'iOS 10.2.1', 'iOS 7.1.2', 'Android 4.0.4',
       'Mac OS X 10.7.5', 'Chrome OS 9901.77', 'Chrome OS 10323.67',
       'Chrome OS 10452.96', 'Other ', 'iOS 8.1.1', 'iOS 11.0.2

In [8]:
checked = ['windows', 'android', 'linux', 'mac', 'ios', 'ubuntu', 'chrome os', 'tizen', 'blackberry', 'other']
os_version_parsed = df['operating_system_version'].dropna().map(lambda x: x.lower())

def find_os(os_version):
    for os in checked:
        if os in os_version:
            return os
    return "another"


df['operating_system'] = os_version_parsed.map(find_os)
df['operating_system'] = df['operating_system'].astype('category')

In [9]:
# Chequeamos cuantos os quedaron con el nombre 'another' (idealmente, ninguno)
df[['operating_system_version', 'operating_system']].dropna().head(10)
df[df['operating_system'] == 'another'][['operating_system_version', 'operating_system']].head(10)

,operating_system_version,operating_system
2139528,Fedora,another
2139529,Fedora,another
2141817,FreeBSD,another
2260955,Fedora,another
2260956,Fedora,another
2267906,Symbian OS,another


In [10]:
df['operating_system'].value_counts()

android       96901
windows       96531
ios            8312
mac             902
linux           766
ubuntu          230
chrome os       205
other           105
blackberry       74
tizen            37
another           6
Name: operating_system, dtype: int64

In [11]:
df[['operating_system_version', 'operating_system']].dropna().drop_duplicates().to_csv('data/os.csv', index=False)

In [12]:
os_csv = pd.read_csv("data/os.csv", low_memory=False)
display(os_csv.head())
display(os_csv.shape)

,operating_system_version,operating_system
0,Android 5.0.2,android
1,Ubuntu,ubuntu
2,Android 7,android
3,Android 6.0.1,android
4,Windows 7,windows


(131, 2)

## browsers.csv

Exploradores de internet desde los cuales se accedió al sitio

In [13]:
df['browser_version'].unique().tolist()

[nan,
 'Chrome Mobile 66.0',
 'Firefox 57',
 'Chrome 66.0',
 'Chrome Mobile 34.0',
 'Chrome Mobile 65.0',
 'Facebook 172',
 'Facebook 173',
 'Mobile Safari 11',
 'Chrome 65.0',
 'Chrome 67.0',
 'Safari 11.1',
 'Firefox 60',
 'Firefox 59',
 'IE 11',
 'Chrome Mobile 39',
 'Chrome Mobile 56.0',
 'Chrome 63.0',
 'Chrome 64.0',
 'Chrome Mobile 64.0',
 'Mobile Safari 9',
 'Chrome 62.0',
 'Facebook 166',
 'Facebook 163',
 'Chrome Mobile 63.0',
 'Chrome 61.0',
 'Chrome 55.0',
 'Chrome 51.0',
 'Chrome Mobile 61.0',
 'Chrome Mobile 55.0',
 'Samsung Internet 6.4',
 'Opera 52.0',
 'Chrome 49.0',
 'Edge 13.10586',
 'Facebook 94',
 'Android 5.1',
 'Chrome Mobile 62.0',
 'Opera 53.0',
 'Edge 17.17134',
 'Facebook 91',
 'Facebook 92',
 'Facebook 88',
 'Facebook 89',
 'Chrome 68.0',
 'Edge 16.16299',
 'Chrome Mobile 50.0',
 'Chrome Mobile 58.0',
 'Chrome 56.0',
 'Mobile Safari 10',
 'Chrome 60',
 'Chrome Mobile 46.0',
 'Firefox 54',
 'Facebook 171',
 'IE Mobile 11',
 'Chrome 57.0',
 'Chrome Mobile iOS 

In [14]:
checked = ['mobile safari', 'chrome mobile', 'ie mobile', 'firefox mobile', 'edge mobile', 'opera mobile',
           'mobile', 'blackberry os', 'blackberry webkit', 'chrome', 'android', 'opera', 'ie', 'firefox', 
           'facebook', 'samsung', 'chromium', 'edge', 'yandex', 'uc', 
           'other', 'safari', 'puffin', 'maxthon', 'vivaldi', 'pinterest']
browser_version_parsed = df['browser_version'].dropna().map(lambda x: x.lower())

def find_browser(browser_version):
    for browser in checked:
        if browser in browser_version:
            return browser
    return "other"


df['browser'] = browser_version_parsed.map(find_browser)
df['browser'] = df['browser'].astype('category')

In [15]:
df[['browser_version', 'browser']].dropna()
display(df['browser'].value_counts().head())
df[['browser_version', 'browser']].head()
df[df['browser'] == 'other']['browser_version']

chrome           87918
chrome mobile    85915
mobile safari     7730
firefox           6777
samsung           4809
Name: browser, dtype: int64

2143535    WebKit Nightly 535.20
2172409              K-Meleon 76
2203820                    Other
2239081                    Other
2254099    WebKit Nightly 535.20
2277183                    Other
2277184                    Other
2277185                    Other
2277186                    Other
2283962                    Other
2286442                    Other
2331163    WebKit Nightly 535.20
2337195                    Other
Name: browser_version, dtype: object

In [16]:
df[['browser_version', 'browser']].dropna().drop_duplicates().to_csv('data/browsers.csv', index=False)

In [17]:
browsers_csv = pd.read_csv("data/browsers.csv", low_memory=False)
display(browsers_csv.head())
display(browsers_csv.shape)

,browser_version,browser
0,Chrome Mobile 66.0,chrome mobile
1,Firefox 57,firefox
2,Chrome 66.0,chrome
3,Chrome Mobile 34.0,chrome mobile
4,Chrome Mobile 65.0,chrome mobile


(366, 2)

## sessions.csv

Eventos por usuario

Definimos una sesión como una serie de eventos por usuario, los cuales están todos con menos de 30 minutos de inactividad entre el actual y el anterior.

In [18]:
funnel = df

funnel = funnel.sort_values(['person', 'timestamp'])

funnel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,brand,operating_system,browser
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,samsung,NaN,NaN
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,Brazil,Computer,1920x1080,Windows 10,Chrome 66.0,NaN,windows,chrome
1507716,2018-05-17 13:45:00,checkout,0008ed71,NaN,8247.0,iPhone SE,Bom,64GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iphone,NaN,NaN
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,Brazil,Computer,1920x1080,Windows 10,Chrome 66.0,NaN,windows,chrome
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
funnel['time_diff'] = funnel.groupby('person')['timestamp'].diff()
funnel['time_diff'] = funnel['time_diff'].fillna(0)
funnel['time_diff_min'] = funnel['time_diff'] / np.timedelta64(1, 'm')

In [20]:
funnel[['person', 'event', 'timestamp', 'time_diff', 'time_diff_min']].head()

,person,event,timestamp,time_diff,time_diff_min
1507286,0008ed71,checkout,2018-05-17 12:27:47,00:00:00,0.000000
2336760,0008ed71,visited site,2018-05-17 13:44:59,01:17:12,77.200000
1507716,0008ed71,checkout,2018-05-17 13:45:00,00:00:01,0.016667
2336761,0008ed71,visited site,2018-05-17 16:21:54,02:36:54,156.900000
2122051,0008ed71,generic listing,2018-05-17 16:22:06,00:00:12,0.200000


In [21]:
THRESHOLD = 30 # minutos

funnel['new_session'] = funnel['time_diff_min'] > THRESHOLD
funnel['session_id'] = funnel.groupby('person')['new_session'].cumsum()
funnel['session_id'] = funnel['session_id'].astype('int')

In [22]:
funnel[['person', 'new_session', 'session_id']].head(8)

,person,new_session,session_id
1507286,0008ed71,False,0
2336760,0008ed71,True,1
1507716,0008ed71,False,1
2336761,0008ed71,True,2
2122051,0008ed71,False,2
1505383,0008ed71,False,2
2146920,00091926,False,0
121981,00091926,False,0


In [23]:
gb = funnel.groupby(['person', 'session_id'])


funnel['session_cumno'] = gb.cumcount()                        
funnel['session_total_events'] = pd.Series(np.repeat(gb.size(), gb.size().values)).values
funnel['session_first'] = funnel['session_cumno'] == 0
funnel['session_last'] = funnel['session_cumno'] == (-1+funnel['session_total_events'])

In [24]:
cols = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'event', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last']

funnel[cols].head()

,person,timestamp,time_diff_min,session_id,event,session_total_events,session_cumno,session_first,session_last
1507286,0008ed71,2018-05-17 12:27:47,0.000000,0,checkout,1,0,True,True
2336760,0008ed71,2018-05-17 13:44:59,77.200000,1,visited site,2,0,True,False
1507716,0008ed71,2018-05-17 13:45:00,0.016667,1,checkout,2,1,False,True
2336761,0008ed71,2018-05-17 16:21:54,156.900000,2,visited site,3,0,True,False
2122051,0008ed71,2018-05-17 16:22:06,0.200000,2,generic listing,3,1,False,False


In [25]:
funnel['is_conversion'] = funnel['event'] == 'conversion'
gb = funnel.groupby(['person', 'session_id'])['is_conversion']

funnel['session_total_conversions'] = pd.Series(np.repeat(gb.sum(), gb.size().values)).values
funnel['session_has_conversion'] = funnel['session_total_conversions'] > 0

funnel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,time_diff_min,new_session,session_id,session_cumno,session_total_events,session_first,session_last,is_conversion,session_total_conversions,session_has_conversion
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,0.000000,False,0,0,1,True,True,False,0.0,False
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,77.200000,True,1,0,2,True,False,False,0.0,False
1507716,2018-05-17 13:45:00,checkout,0008ed71,NaN,8247.0,iPhone SE,Bom,64GB,Cinza espacial,NaN,...,0.016667,False,1,1,2,False,True,False,0.0,False
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,156.900000,True,2,0,3,True,False,False,0.0,False
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,0.200000,False,2,1,3,False,False,False,0.0,False


In [26]:
funnel['is_checkout'] = funnel['event'] == 'checkout'
gb = funnel.groupby(['person', 'session_id'])['is_checkout']
funnel['session_total_checkouts'] = pd.Series(np.repeat(gb.sum(), gb.size().values)).values

funnel['session_has_checkout'] = funnel['session_total_checkouts'] > 0

funnel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,session_cumno,session_total_events,session_first,session_last,is_conversion,session_total_conversions,session_has_conversion,is_checkout,session_total_checkouts,session_has_checkout
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,0,1,True,True,False,0.0,False,True,1.0,True
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,2,True,False,False,0.0,False,False,1.0,True
1507716,2018-05-17 13:45:00,checkout,0008ed71,NaN,8247.0,iPhone SE,Bom,64GB,Cinza espacial,NaN,...,1,2,False,True,False,0.0,False,True,1.0,True
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,3,True,False,False,0.0,False,False,1.0,True
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,1,3,False,False,False,0.0,False,False,1.0,True


In [27]:
funnel['ad_origin'] = (funnel['event'] == 'ad campaign hit') & ((funnel['session_first']) | (funnel['session_cumno'] == 1))
gb = funnel.groupby(['person', 'session_id'])['ad_origin']

funnel['session_ad'] = pd.Series(np.repeat(gb.sum().values, gb.size().values)).values
funnel['session_ad'] = funnel['session_ad'] > 0

funnel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,session_first,session_last,is_conversion,session_total_conversions,session_has_conversion,is_checkout,session_total_checkouts,session_has_checkout,ad_origin,session_ad
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,True,True,False,0.0,False,True,1.0,True,False,False
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,0.0,False,False,1.0,True,False,False
1507716,2018-05-17 13:45:00,checkout,0008ed71,NaN,8247.0,iPhone SE,Bom,64GB,Cinza espacial,NaN,...,False,True,False,0.0,False,True,1.0,True,False,False
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,False,0.0,False,False,1.0,True,False,False
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,False,False,False,0.0,False,False,1.0,True,False,False


In [28]:
gb = funnel.groupby(['person', 'session_id'])['timestamp']

funnel['session_timestamp_first'] = pd.Series(np.repeat(gb.min().values, gb.size().values)).values
funnel['session_timestamp_last'] = pd.Series(np.repeat(gb.max().values, gb.size().values)).values

funnel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,is_conversion,session_total_conversions,session_has_conversion,is_checkout,session_total_checkouts,session_has_checkout,ad_origin,session_ad,session_timestamp_first,session_timestamp_last
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,False,0.0,False,True,1.0,True,False,False,2018-05-17 12:27:47,2018-05-17 12:27:47
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,0.0,False,False,1.0,True,False,False,2018-05-17 13:44:59,2018-05-17 13:45:00
1507716,2018-05-17 13:45:00,checkout,0008ed71,NaN,8247.0,iPhone SE,Bom,64GB,Cinza espacial,NaN,...,False,0.0,False,True,1.0,True,False,False,2018-05-17 13:44:59,2018-05-17 13:45:00
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,0.0,False,False,1.0,True,False,False,2018-05-17 16:21:54,2018-05-17 16:28:37
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,False,0.0,False,False,1.0,True,False,False,2018-05-17 16:21:54,2018-05-17 16:28:37


In [29]:
funnel[cols].head()

,person,timestamp,time_diff_min,session_id,event,session_total_events,session_cumno,session_first,session_last
1507286,0008ed71,2018-05-17 12:27:47,0.000000,0,checkout,1,0,True,True
2336760,0008ed71,2018-05-17 13:44:59,77.200000,1,visited site,2,0,True,False
1507716,0008ed71,2018-05-17 13:45:00,0.016667,1,checkout,2,1,False,True
2336761,0008ed71,2018-05-17 16:21:54,156.900000,2,visited site,3,0,True,False
2122051,0008ed71,2018-05-17 16:22:06,0.200000,2,generic listing,3,1,False,False


In [30]:
cols_csv = ['time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last', \
        'session_total_conversions', 'session_has_conversion', \
        'session_total_checkouts', 'session_has_checkout', 'session_ad',
        'session_timestamp_first', 'session_timestamp_last']

funnel = funnel.sort_index()

funnel[cols].head()

,person,timestamp,time_diff_min,session_id,event,session_total_events,session_cumno,session_first,session_last
0,4886f805,2018-05-18 00:11:59,0.050000,0,viewed product,9,7,False,False
1,ad93850f,2018-05-18 00:11:27,0.016667,2,viewed product,31,2,False,False
2,0297fc1e,2018-05-18 00:11:16,0.116667,78,viewed product,2,1,False,True
3,2d681dd8,2018-05-18 00:11:14,0.616667,0,viewed product,10,5,False,False
4,cccea85e,2018-05-18 00:11:09,0.066667,11,viewed product,65,55,False,False


In [31]:
cols_check_ord = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last', \
        'session_total_conversions', 'session_has_conversion', \
        'session_total_checkouts', 'session_has_checkout', 'session_ad',
        'session_timestamp_first', 'session_timestamp_last']

funnel.sort_values(['person', 'timestamp'])[cols_check_ord].head()

,person,timestamp,time_diff_min,session_id,session_total_events,session_cumno,session_first,session_last,session_total_conversions,session_has_conversion,session_total_checkouts,session_has_checkout,session_ad,session_timestamp_first,session_timestamp_last
1507286,0008ed71,2018-05-17 12:27:47,0.000000,0,1,0,True,True,0.0,False,1.0,True,False,2018-05-17 12:27:47,2018-05-17 12:27:47
2336760,0008ed71,2018-05-17 13:44:59,77.200000,1,2,0,True,False,0.0,False,1.0,True,False,2018-05-17 13:44:59,2018-05-17 13:45:00
1507716,0008ed71,2018-05-17 13:45:00,0.016667,1,2,1,False,True,0.0,False,1.0,True,False,2018-05-17 13:44:59,2018-05-17 13:45:00
2336761,0008ed71,2018-05-17 16:21:54,156.900000,2,3,0,True,False,0.0,False,1.0,True,False,2018-05-17 16:21:54,2018-05-17 16:28:37
2122051,0008ed71,2018-05-17 16:22:06,0.200000,2,3,1,False,False,0.0,False,1.0,True,False,2018-05-17 16:21:54,2018-05-17 16:28:37


In [32]:
funnel[cols_csv].to_csv('data/sessions.csv', index=False)

In [33]:
sessions_csv = pd.read_csv("data/sessions.csv", low_memory=False)
display(sessions_csv.head())
display(sessions_csv.shape)

,time_diff_min,session_id,session_total_events,session_cumno,session_first,session_last,session_total_conversions,session_has_conversion,session_total_checkouts,session_has_checkout,session_ad,session_timestamp_first,session_timestamp_last
0,0.050000,0,9,7,False,False,0.0,False,1.0,True,False,2018-05-18 00:07:22,2018-05-18 00:30:30
1,0.016667,2,31,2,False,False,0.0,False,0.0,False,True,2018-05-18 00:11:26,2018-05-18 00:23:33
2,0.116667,78,2,1,False,True,0.0,False,0.0,False,False,2018-05-18 00:11:09,2018-05-18 00:11:16
3,0.616667,0,10,5,False,False,0.0,False,0.0,False,False,2018-05-18 00:08:29,2018-05-18 00:11:25
4,0.066667,11,65,55,False,False,0.0,False,0.0,False,False,2018-05-17 23:52:57,2018-05-18 00:12:50


(2341681, 13)

## prices.csv

In [2]:
mapping_conditions = {'Bom':15,
                      'Muito Bom': 6,
                      'Excelente': 5,
                      'Bom - Sem Touch ID':17,
                      'Novo':18,
                       np.nan:0
                     }

df_tmp = df.loc[df['model'].notnull()][['sku','model','color','storage','condition']]
df_tmp['to_search'] = df_tmp['model'] + ' ' + df_tmp['storage'].astype(str) + ' ' + df_tmp['color']
df_tmp['condition_n'] = df_tmp['condition'].transform(lambda x: mapping_conditions[x])
df_tmp['to_search+condition_n'] = df_tmp['to_search'] + ' ' + df_tmp['condition_n'].astype(str)

df_tmp = df_tmp.drop_duplicates()
df_tmp = df_tmp.dropna()
display(df_tmp.head())

dic_modelos_condiciones = {}
modelos = df_tmp['to_search'].tolist()
condiciones = df_tmp['condition_n'].tolist()
for m,c in zip(modelos,condiciones):
    if m in dic_modelos_condiciones:
        dic_modelos_condiciones[m].append(c)
    else: dic_modelos_condiciones[m] = [c]


,sku,model,color,storage,condition,to_search,condition_n,to_search+condition_n
0,9288.0,Samsung Galaxy J7 Prime,Dourado,32GB,Excelente,Samsung Galaxy J7 Prime 32GB Dourado,5,Samsung Galaxy J7 Prime 32GB Dourado 5
1,304.0,iPhone 5s,Cinza espacial,32GB,Muito Bom,iPhone 5s 32GB Cinza espacial,6,iPhone 5s 32GB Cinza espacial 6
2,6888.0,iPhone 6S,Prateado,64GB,Muito Bom,iPhone 6S 64GB Prateado,6,iPhone 6S 64GB Prateado 6
3,11890.0,iPhone 7,Vermelho,128GB,Bom,iPhone 7 128GB Vermelho,15,iPhone 7 128GB Vermelho 15
4,7517.0,LG G4 H818P,Branco,32GB,Excelente,LG G4 H818P 32GB Branco,5,LG G4 H818P 32GB Branco 5


In [5]:
def request_soup(url):
    try:
        req = requests.get(url)
        html = req.content
        soup = BeautifulSoup(html, "lxml")
    except requests.exceptions.ConnectionError:
        raise ConnectionError(f"Connection with {url} refused.")
    return soup

BASE = 'https://www.trocafone.com/comprar/list?'
MODELO = 'q'
CONDICION = 'condition'

precios = {}
i=1
for modelo, condiciones in dic_modelos_condiciones.items(): 
    print(f'Iteración {i} de {len(dic_modelos_condiciones)}:')
    modelo_s = modelo.replace(' ','+')
    for condicion in condiciones:
        url = f'{BASE}{MODELO}={modelo_s}&{CONDICION}={condicion}'
        print(f'\t * {url}: ',end='')
        soup = request_soup(url)
        precio = soup.find(class_='price-value')
        if precio and precio.string:
            precio = precio.string
        else: precio = 0
        precio.replace('.','').replace(',','.') # (pasar de 1.2900,00 a 12900.00, para que Python se lleve bien con el float)
        print(f'${precio}')
        precios[f'{modelo} {condicion}'] = precio
    i+=1

Iteración 1 de 810:
	 * https://www.trocafone.com/comprar/list?q=Samsung+Galaxy+J7+Prime+32GB+Dourado&condition=5: $699,00
	 * https://www.trocafone.com/comprar/list?q=Samsung+Galaxy+J7+Prime+32GB+Dourado&condition=6: $2.569,00
	 * https://www.trocafone.com/comprar/list?q=Samsung+Galaxy+J7+Prime+32GB+Dourado&condition=15: $2.229,00
Iteración 2 de 810:
	 * https://www.trocafone.com/comprar/list?q=iPhone+5s+32GB+Cinza+espacial&condition=6: $879,00
	 * https://www.trocafone.com/comprar/list?q=iPhone+5s+32GB+Cinza+espacial&condition=5: $939,00
	 * https://www.trocafone.com/comprar/list?q=iPhone+5s+32GB+Cinza+espacial&condition=17: $949,00
	 * https://www.trocafone.com/comprar/list?q=iPhone+5s+32GB+Cinza+espacial&condition=15: $779,00
Iteración 3 de 810:
	 * https://www.trocafone.com/comprar/list?q=iPhone+6S+64GB+Prateado&condition=6: $1.779,00
	 * https://www.trocafone.com/comprar/list?q=iPhone+6S+64GB+Prateado&condition=5: 

KeyboardInterrupt: Proceso muy costoso en tiempo. Se corta la ej

In [ ]:
orden = df_tmp['to_search+condition_n']

precios_ordenado = []

for modelo in orden:
    precios_ordenado.append(precios[modelo])

df_tmp['precio_reales'] = precios_ordenado

In [ ]:
df_tmp[['sku','precio_reales']].to_csv('data/prices.csv', index=False)

In [6]:
prices_csv = pd.read_csv("data/prices.csv", low_memory=False)
display(prices_csv.head())
display(prices_csv.shape)

,sku,precio_reales
0,9288.0,749.0
1,304.0,1679.0
2,6888.0,2329.0
3,11890.0,2469.0
4,7517.0,1939.0


(2328, 2)